# ARMA(p,q) and ARIMA(p,d,q)

In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARMA,ARMAResults,ARIMA,ARIMAResults

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
# Load datasets
df1 = pd.read_csv(r'C:/Users/User/Downloads/UDEMY_TSA_FINAL/UDEMY_TSA_FINAL/Data/DailyTotalFemaleBirths.csv',index_col='Date',parse_dates=True)

df2 = pd.read_csv(r'C:/Users/User/Downloads/UDEMY_TSA_FINAL/UDEMY_TSA_FINAL/Data/TradeInventories.csv',index_col='Date',parse_dates=True)


In [ ]:
df1.plot()
#stationary data

In [ ]:
df2.plot()

# Dickey-Fuller Test

To determine if an incoming time series is stationary, let's write a function that performs the augmented Dickey-Fuller Test.


In [4]:
from statsmodels.tsa.stattools import adfuller
def adf_test(dataset):
    result = adfuller(df1.dropna(), autolag ='AIC')
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)
    for key,val in result[4].items():
        out[f'critical value ({key})']=val
        
    print(out.to_string())
    if result[1]<0.05:
        print(out['p-value'])
        print("data is stationary")
        print("reject Null Hypothesis")
        print("It has no unit root")
    else:
        print(out['p-value'])
        print("data is stationary")
        print("rejetc Null Hypothesis")
        print("It has no unit root")
    

In [ ]:
adf_test(df1['Births'])

In [ ]:
#let us take first 4 months of birth rates
df1['Births'].plot(figsize=(12,5));

In [ ]:
#Using pmdarima.auto_arima, determine the(p,q) orders , ARMA
from pmdarima import auto_arima
auto_arima(df1['Births'], seasonal=False, m=4).summary()

In [ ]:
#if we will check the auto_Arima() of step_wise function, we get P,D,Q and m terms are zero.Because it is a Seasonal model

In [ ]:
stepwise_fit = auto_arima(df1['Births'], start_p=0, start_q=0,
                          max_p=6, max_q=3, m=4,
                          seasonal=False,
                          d=None, trace=True,
                          error_action='ignore',   # we don't want to know if an order does not work
                          suppress_warnings=True,  # we don't want convergence warnings
                          stepwise=True) 

# #Split the data in triain and test sets


In [ ]:
train = df1.iloc[:90]
test = df1.iloc[90:] 

In [ ]:
train.shape,test.shape, df1.shape

In [ ]:
train

# Fit an ARMA(p,q) Model

In [9]:
import statsmodels.api as sm

In [ ]:
model =  sm.tsa.arima.ARIMA(train['Births'],order=(1,1,1))

In [ ]:
results = model.fit()
results.summary()

In [ ]:
#now calculate a month's predicted values

In [ ]:
start = len(train)
end = len(train)+len(test)-1

In [ ]:
start,len(test),end

In [ ]:
predictions = results.predict(start,end).rename('ARIMA(1,1,1)PREDICTIONS')

In [ ]:
df1.tail()

In [ ]:
predictions
#Last 12 months prediction values

# plot Predictions against true values for last 12 mionths data

In [ ]:
title = 'Daily Total Female Births'
ylabel='Births'
xlabel=''

ax = test['Births'].plot(legend=True, figsize=(12,6), title = title)
predictions.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel=xlabel, ylabel=ylabel)

In [ ]:
test.mean()

In [ ]:
predictions.mean()

In [ ]:
#The above result shows quite close to each other.  In the next section we'll take additional steps to evaluate the performance of our predictions, and forecast into the future.


# Seasonal model